THIS NOTEBOOK IS USED TO TEST THE MODULAR CODES.
<br>IF THE CODES ARE WORKING PROPERLY THEN WE UPDATE THE CORR. FILES(CONFIG,ENTITY,etc)

In [2]:
import os

In [3]:
%pwd
# command to check which directory is in use

'c:\\Users\\LENOVO\\Desktop\\Chicken Disease Classifier\\Chicken-Disease-Classification\\research'

In [4]:
os.chdir("../")
%pwd

'c:\\Users\\LENOVO\\Desktop\\Chicken Disease Classifier\\Chicken-Disease-Classification'

In [5]:
# Now we are creating custom return type using entity
from dataclasses import dataclass
from pathlib import Path

# this is the custom datatype
# these are fetched from config.yaml
# frozen=True means the objects of the class can't be modified
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [7]:

# this is creating directories by getting data from config.yaml file
# the object of this class is storing the -> config file path and -> params file path 
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    # Here we are using the custom datatype that we have created earlier
    def get_data_ingestion_config(self) -> DataIngestionConfig:

        # config stores the list:- [root_dir, source_URL, local_data_file, unzip_dir]
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
# Updating the components
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    # this function downloads the zip file(dataset) from the url and is stored in the local_data_file directory
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    # this function extracts the data from the zip file directory
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
# Code works fine and the directories are created as needed
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-13 23:32:13,082: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-13 23:32:13,086: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-13 23:32:13,089: INFO: common: created directory at: artifacts]
[2024-04-13 23:32:13,090: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-13 23:32:31,324: INFO: 1138818090: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "871f8d604aca86a2a474c9ddd5c478a7a830df94e0c10769e0c2834e22caef27"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7A8E:3BF0E4:9416E:C2024:661AC8A6
Accept-Ranges: bytes
Date: Sat, 13 Apr 2024 18:02:15 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10234-MAA
X-Cache: MI